<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/grad_cam_cnn_vit_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ---------------------------
# full_script_with_gradcam_v3.py
# ---------------------------
import os
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import train_test_split

# ------------------------
# USER SETTINGS (change if needed)
# ------------------------
dataset_dir = "/kaggle/input/alz-b-1100/alzheimer_new_11/alzheimer_new"  # <--- update path if needed
img_size = (128, 128)
expected_classes = 8    # number of classes in your dataset
patch_size = 16
projection_dim = 64
transformer_layers = 4
num_heads = 8
batch_size = 32
epochs = 5               # set to 100 for full training
num_examples = 8         # number of validation examples to save Grad-CAM for

# ------------------------
# LOAD DATA
# ------------------------
X = []
y = []

class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
print("✅ Classes found:", class_names)
if len(class_names) != expected_classes:
    raise ValueError(f"Expected exactly {expected_classes} classes, found {len(class_names)}: {class_names}")

for idx, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_dir, class_name)
    print(f"🔍 Processing class '{class_name}' ({idx})")
    for root, _, files in os.walk(class_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img, dtype=np.float32) / 255.0  # normalized to [0,1]
                if img_array.shape == (img_size[0], img_size[1], 3):
                    X.append(img_array)
                    y.append(idx)
            except Exception as e:
                print(f"⚠️ Failed to process {file_path}: {e}")

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)
print(f"\n✅ Dataset loaded: X={X.shape}, y={y.shape}")

# ------------------------
# TRAIN/VAL SPLIT
# ------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# ------------------------
# CUSTOM LAYERS (serializable)
# ------------------------
@register_keras_serializable()
class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(Patches, self).__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = tf.shape(patches)[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

@register_keras_serializable()
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)

    def call(self, patches):
        batch = tf.shape(patches)[0]
        seq_len = tf.shape(patches)[1]
        positions = tf.range(start=0, limit=seq_len, delta=1)
        pos_embeddings = self.position_embedding(positions)
        proj = self.projection(patches)
        return proj + pos_embeddings

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches, "projection_dim": self.projection_dim})
        return config

# ------------------------
# MODEL BUILD (fixed GRU input shape)
# ------------------------
def build_hybrid_model(input_shape=(128,128,3), patch_size=16, projection_dim=64, transformer_layers=4, num_heads=8, expected_classes=8):
    inputs = layers.Input(shape=input_shape)

    # CNN branch
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x_cnn = layers.Flatten()(x)

    # ViT branch
    patches = Patches(patch_size)(inputs)
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    encoded = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        # Self-attention block
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded)
        attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
        x2 = layers.Add()([attn_out, encoded])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        encoded = layers.Add()([x3, x2])

    # Project tokens to fixed feature size for GRU (no None dims)
    token_features = layers.Dense(256, activation='relu')(encoded)   # shape: (batch, num_patches, 256)
    vit_out = layers.GRU(256)(token_features)                        # GRU input fully-defined

    # Merge and classify
    merged = layers.concatenate([x_cnn, vit_out])
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.Dropout(0.3)(merged)
    outputs = layers.Dense(expected_classes, activation='softmax')(merged)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Build and compile
model = build_hybrid_model(input_shape=(img_size[0], img_size[1], 3),
                           patch_size=patch_size,
                           projection_dim=projection_dim,
                           transformer_layers=transformer_layers,
                           num_heads=num_heads,
                           expected_classes=expected_classes)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------
# TRAIN (adjust epochs as you like)
# ------------------------
start_time = time.time()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)
print("Training finished. Time:", time.time() - start_time)

# ------------------------
# GRAD-CAM UTILITIES
# ------------------------
def find_last_conv_layer(model):
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
    return None

def make_gradcam_heatmap(model, img_array, last_conv_layer_name, class_index=None):
    """
    img_array: HxWxC in [0,1]
    returns: heatmap HxW normalized 0..1
    """
    img_tensor = tf.expand_dims(img_array, axis=0)
    last_conv = model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model([model.inputs], [last_conv.output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        if class_index is None:
            class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-9)
    return heatmap.numpy()

def overlay_gradcam_on_image(image, heatmap, alpha=0.4):
    heatmap_uint8 = np.uint8(255 * heatmap)
    jet = plt.cm.jet(heatmap_uint8 / 255.0)[:, :, :3]
    jet_resized = tf.image.resize(jet, (image.shape[0], image.shape[1])).numpy()
    overlay = image * (1 - alpha) + jet_resized * alpha
    overlay = np.clip(overlay, 0, 1)
    return overlay

# ------------------------
# GUIDED BACKPROP (guided relu)
# ------------------------
@tf.custom_gradient
def guided_relu(x):
    def grad(dy):
        return tf.cast(dy > 0, "float32") * tf.cast(x > 0, "float32") * dy
    return tf.nn.relu(x), grad

def make_guided_model(original_model):
    """
    Clone the original architecture, load weights, then replace relu activations with guided_relu.
    We avoid model.from_config to prevent custom-layer serialization issues.
    """
    # Rebuild model via call to build_hybrid_model (same signature)
    guided = build_hybrid_model(input_shape=(img_size[0], img_size[1], 3),
                                patch_size=patch_size,
                                projection_dim=projection_dim,
                                transformer_layers=transformer_layers,
                                num_heads=num_heads,
                                expected_classes=expected_classes)
    guided.set_weights(original_model.get_weights())

    # Replace relu activations on layers that have activation attribute
    for layer in guided.layers:
        if hasattr(layer, "activation") and layer.activation == tf.nn.relu:
            layer.activation = guided_relu

    # Also replace Activation layers if present
    for i, layer in enumerate(guided.layers):
        if isinstance(layer, tf.keras.layers.Activation):
            if layer.activation == tf.nn.relu:
                guided.layers[i] = tf.keras.layers.Activation(guided_relu)

    return guided

# Build guided model
guided_model = make_guided_model(model)

def guided_backprop(guided_model, img_array, class_index=None):
    """
    Returns guided backpropagation saliency (HxWx3) in [0,1]
    """
    img_tensor = tf.expand_dims(img_array, axis=0)
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = guided_model(img_tensor)
        if class_index is None:
            class_index = tf.argmax(preds[0])
        loss = preds[:, class_index]

    grads = tape.gradient(loss, img_tensor)[0].numpy()
    # Keep positive gradients (guided backprop behavior)
    grads = np.maximum(grads, 0.0)
    # normalize per image
    denom = np.max(grads) - np.min(grads) + 1e-9
    grads = (grads - np.min(grads)) / denom
    return grads

# ------------------------
# RUN & SAVE VISUALIZATIONS (Grad-CAM, Guided BP, Guided Grad-CAM)
# ------------------------
out_dir = "gradcam_outputs"
os.makedirs(out_dir, exist_ok=True)

last_conv_name = find_last_conv_layer(model)
if last_conv_name is None:
    raise RuntimeError("No Conv2D found for Grad-CAM target. Check model.")

print("Using Conv layer for Grad-CAM:", last_conv_name)

num_examples = min(num_examples, X_val.shape[0])
indices = np.random.choice(range(X_val.shape[0]), size=num_examples, replace=False)

for idx in indices:
    img = X_val[idx]
    true_label = int(y_val[idx])
    preds = model.predict(np.expand_dims(img, 0))
    pred_label = int(np.argmax(preds[0]))

    # Grad-CAM
    heatmap = make_gradcam_heatmap(model, img, last_conv_name, class_index=pred_label)
    heatmap_resized = tf.image.resize(heatmap[..., None], (img.shape[0], img.shape[1])).numpy()[...,0]
    overlay = overlay_gradcam_on_image(img, heatmap_resized, alpha=0.45)

    # Guided Backprop (on guided_model)
    gb = guided_backprop(guided_model, img, class_index=pred_label)  # HxWx3

    # Guided Grad-CAM: multiply per-channel guided backprop by single-channel heatmap
    guided_gc = gb * np.expand_dims(heatmap_resized, axis=-1)
    # Normalize guided_gradcam to [0,1] for visualization
    guided_gc = (guided_gc - guided_gc.min()) / (guided_gc.max() - guided_gc.min() + 1e-9)

    # Save figure
    fig, axs = plt.subplots(1, 4, figsize=(16, 4))
    axs[0].imshow(img); axs[0].set_title(f"Original\ntrue={true_label}"); axs[0].axis('off')
    axs[1].imshow(heatmap_resized, cmap='jet'); axs[1].set_title(f"Grad-CAM\npred={pred_label}"); axs[1].axis('off')
    axs[2].imshow(overlay); axs[2].set_title("Grad-CAM Overlay"); axs[2].axis('off')
    axs[3].imshow(guided_gc); axs[3].set_title("Guided Grad-CAM"); axs[3].axis('off')
    plt.tight_layout()

    out_path = os.path.join(out_dir, f"viz_idx{idx}_pred{pred_label}_true{true_label}.png")
    fig.savefig(out_path, dpi=200)
    plt.close(fig)
    print("Saved:", out_path)

print("✅ All visualizations saved to:", out_dir)
